In [ ]:
import numpy as np

data = np.load("./../data/collected_data.npy", allow_pickle=True)


In [ ]:
import torch
print(torch.cuda.device_count())


In [ ]:
len(data)

In [ ]:
data[0].shape

In [ ]:
data[0][0]

In [ ]:
# Take first episode's data
episode_data = data[0]

# Extract the states (images)
states = [x[0] for x in episode_data]

# Take some samples (e.g., first 5)
sample_states = states[0:999]


In [ ]:
import matplotlib.pyplot as plt

cols = 40
for idx, image in enumerate(sample_states):
    # plot with 10 columns and number of rows based on number of samples
    plt.subplot(int(np.ceil(len(sample_states)/cols)), cols, idx+1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    # add some vertical space between the images
    plt.subplots_adjust(hspace=0.6)
    # label the axes with the action
    plt.title(np.round(episode_data[idx][1],2), fontsize=3)
    #make the length of the figure relative to the number of samples
    plt.gcf().set_size_inches(20, len(sample_states)/cols)
    

plt.show()


In [ ]:
import pandas as pd

flattened_data = []
episode_count = 1

for episode in data:
    for step in episode:
        # Convert step to list and prepend episode_count
        step_list = [episode_count] + list(step)
        flattened_data.append(step_list)
    episode_count += 1

df = pd.DataFrame(
    flattened_data,
    columns=["Episode", "State", "Action", "Reward", "Done", "Truncated"]
)




In [ ]:
print(df.head())
print(df.describe())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df['Reward'].values, bins=100, kde=True)
plt.show()


In [1]:
import os
print(os.getcwd())
import sys
sys.path.append("../..")


c:\Users\aswegen.d\Dropbox\0_Buas\BuasDev\World Models\worldmodels1\notebooks


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from worldmodels1.src.worldmodels1.cnnvae import VAE

vae = VAE()
# Load the state_dict into CPU memory
state_dict = torch.load('../vae2.pth', map_location='cpu')

# Remove 'module.' prefix from state_dict keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load the modified state_dict into the model
vae.load_state_dict(new_state_dict)
vae.eval()

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    vae = vae.to(device)
    print("Using GPU")

In [ ]:
# Take the states (images) from a random episode's data
episode_idx = np.random.randint(len(data))
print(f"Episode {episode_idx} is selected")
episode_data = data[episode_idx]
states = [x[0] for x in episode_data]

# Randomly sample 30 images
#random_indices = np.random.choice(len(states), size=30, replace=False)
# take 30 sequential images
start = np.random.randint(len(states)-30)
random_indices = np.arange(start, start+30)
# take 30 sequential images skip 5 between starting at start
random_indices = np.arange(start, start+30*5, 5)
random_samples = [states[i] for i in random_indices]
random_samples = np.array(random_samples)
# Ensure random_samples has shape [batch_size, height, width]
# Add channel dimension
random_samples = np.expand_dims(random_samples, axis=1)

# Convert to PyTorch tensor and normalize to range [0, 1]
random_samples_tensor = torch.tensor(random_samples, dtype=torch.float32)/255.

# Move tensor to the appropriate device
random_samples_tensor = random_samples_tensor.to(device)

with torch.no_grad():
    reconstructed, mu, logvar = vae(random_samples_tensor)

# Convert tensors back to numpy for plotting
reconstructed = reconstructed.cpu().numpy()
mu = mu.cpu().numpy()
logvar = logvar.cpu().numpy()

# Calculate reconstruction loss (MSE) for each image
reconstruction_loss = np.sum((random_samples/255. - reconstructed)**2, axis=(1, 2, 3))
# Calculate KL divergence loss for each image
kl_loss = -0.5 * np.sum(1 + logvar - np.square(mu) - np.exp(logvar), axis=1)

plt.figure(figsize=(20, 8))

# Plotting original images
plt.subplot(1, 3, 1)
plt.title('Original Images')
grid_img = make_grid(torch.tensor(random_samples), nrow=5)
plt.imshow(np.transpose(grid_img, (1, 2, 0)))

# Plotting reconstructed images
plt.subplot(1, 3, 2)
plt.title('Reconstructed Images')
grid_img_reconstructed = make_grid(torch.tensor(reconstructed), nrow=5)
plt.imshow(np.transpose(grid_img_reconstructed, (1, 2, 0)))

# Calculate position for annotations
img_size = 64  # Assuming each image is 64x64
grid_size = 5  # 5 images in a row
padding = 2  # Default padding in make_grid

# Annotate with Reconstruction loss and KL loss
for i, (rec_loss, kl) in enumerate(zip(reconstruction_loss, kl_loss)):
    row = i // grid_size
    col = i % grid_size
    x_text = col * (img_size + padding)
    y_text = row * (img_size + padding)
    plt.text(x_text + 5, y_text + 5, f"MSE: {rec_loss:.2f}", color='white', fontsize=8, ha='left', va='top')
    plt.text(x_text + 5, y_text + 20, f"KL: {kl:.2f}", color='red', fontsize=8, ha='left', va='top')

# Plotting latent vectors
plt.subplot(1, 3, 3)
plt.title('Latent Vectors')
latent = mu + np.exp(0.5 * logvar) * np.random.randn(*logvar.shape)
scatter = plt.scatter(latent[:, 0], latent[:, 1])

# Annotate each point
for i, (x, y) in enumerate(latent[:, :2]):
    plt.annotate(str(i), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

# Show the plot
plt.show()

# print the mean of the losses
print(f"Mean Reconstruction loss: {np.mean(reconstruction_loss):.2f}")
print(f"Mean KL loss: {np.mean(kl_loss):.2f}")


In [ ]:
# Choose 10 random episodes
num_random_episodes = 10
random_episodes_indices = np.random.choice(len(data), num_random_episodes, replace=False)

# Initialize an empty list to hold states from multiple episodes
all_samples = []

# Loop through the selected random episodes to collect states
for episode_idx in random_episodes_indices:
    episode_data = data[episode_idx]
    states = [x[0] for x in episode_data]
    all_samples.extend(states)

# Convert the list to a NumPy array (assuming each state is a NumPy array)
all_samples = np.array(all_samples)
# Add channel dimension
all_samples = np.expand_dims(all_samples, axis=1)

# The shape of all_samples will be (num_samples, state_shape)
print(f"Collected a total of {len(all_samples)} states from {num_random_episodes} random episodes.")


In [ ]:
# Convert to PyTorch tensor and normalize to range [0, 1]
all_samples_tensor = torch.tensor(all_samples, dtype=torch.float32)/255.

# Move tensor to the appropriate device
all_samples_tensor = all_samples_tensor.to(device)

with torch.no_grad():
    _, mu, logvar = vae(all_samples_tensor)

mu = mu.cpu().numpy()
logvar = logvar.cpu().numpy()
latent = mu + np.exp(0.5 * logvar) * np.random.randn(*logvar.shape)

# Assuming `all_labels` are the corresponding labels for `all_samples`
plt.scatter(latent[:, 0], latent[:, 1])#, c=all_labels, cmap='jet')
plt.colorbar()
plt.title('Colored Latent Space 1st 2 dims')
plt.xlabel('Latent Dim 1')
plt.ylabel('Latent Dim 2')



In [ ]:
print(latent.shape)

In [ ]:
import seaborn as sns

sns.kdeplot(x=latent[:, 0], y=latent[:, 1], cmap="Blues", fill=True, thresh=0.05)
plt.title('Density Plot of Latent Space only 1st 2 dims')


In [ ]:
# Compute the mean and std along each dimension of the latent space
means = np.mean(latent, axis=0)
std_devs = np.std(latent, axis=0)

# Pick the two dimensions with the largest std_devs for visualization
largest_std_dims = np.argsort(std_devs)[-2:]  # Get indices of largest std devs

# Extract the values along the selected dimensions
selected_latent = latent[:, largest_std_dims]

# Create a large figure for the density plot
plt.figure(figsize=(20, 20))

# Plot the density
sns.kdeplot(x=selected_latent[:, 0], y=selected_latent[:, 1], cmap="Blues", fill=True, thresh=0.05)

# Calculate the number of images to overlay (1% in this example)
num_to_overlay = int(len(all_samples) * 0.01)
random_indices = np.random.choice(len(all_samples), num_to_overlay, replace=False)

# Choose random images and their corresponding latent coordinates
random_images = all_samples[random_indices]
random_latent_coords = selected_latent[random_indices]

# Overlay the randomly selected images
scale_factor = 0.05  # Adjust as needed
offset_x, offset_y = -1, -1  # Adjust based on your latent space

for i, img in enumerate(random_images):
    x, y = random_latent_coords[i]
    x += offset_x
    y += offset_y
    plt.imshow(img[0], extent=[x, x + scale_factor, y, y + scale_factor], cmap='gray')  # Assuming grayscale images

plt.title('Density Plot of Latent Space with Overlaid Images')
plt.show()


In [ ]:
latent_dim = 32  # Replace with the actual dimension
image_height = 64  # Replace with the actual image height
image_width = 64  # Replace with the actual image width

latent_action_pairs = np.load('./../data/latent_action_pairs (6).npy')
latent_vectors = latent_action_pairs[:, :latent_dim] 
actions = latent_action_pairs[:, latent_dim:]


print(latent_action_pairs.shape)
print(latent_vectors.shape)

In [ ]:
latent_action_pairs[0:10]

In [ ]:
# Take one latent vector
latent_vector = torch.tensor(latent_vectors[0], dtype=torch.float32).unsqueeze(0).to(device)

# Decode the latent vector
with torch.no_grad():
    decoded_img = vae.decode(latent_vector).squeeze().cpu().numpy()

# If your decoded image is not in the shape you expect, reshape it accordingly
decoded_img = decoded_img.reshape((image_height, image_width))  # Assuming grayscale images

plt.imshow(decoded_img, cmap='gray')  # or cmap='color' if your image is color
plt.title('Decoded Image from Latent Vector')
plt.axis('off')
plt.show()


In [ ]:
from ipywidgets import interact

def decode_and_plot(idx):
    latent_vector = torch.tensor(latent_vectors[idx], dtype=torch.float32).unsqueeze(0).to(device)
    
    # Decode the latent vector
    with torch.no_grad():
        decoded_img = vae.decode(latent_vector).squeeze().cpu().numpy()
    
    decoded_img = decoded_img.reshape((image_height, image_width))
    
    # Create a subplot for original and decoded images
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot the original image
    axes[0].imshow(sample_states[idx], cmap='gray')
    axes[0].set_title(f'Original Image (Index {idx})')
    axes[0].axis('off')
    
    # Plot the decoded image
    axes[1].imshow(decoded_img, cmap='gray')
    axes[1].set_title(f'Decoded Image (Index {idx})')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Create a slider to select the latent vector index
interact(decode_and_plot, idx=(0, len(sample_states) - 1))



# LATENTS ARE SHUFFLED NEED TO FIX - This is fixed there was not shuffle the first VAE model was bad.


In [ ]:
# Create a 10x10 grid for plotting
fig, axes = plt.subplots(15, 15, figsize=(15, 15))

# Flatten the 10x10 grid into a list for easy iteration
axes_flat = axes.flatten()

for i, ax in enumerate(axes_flat):
    #Plot original image in original position
    
    
    ax.imshow(sample_states[i], cmap='gray')
    ax.set_title(f'Img {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Create a 10x10 grid for plotting
fig, axes = plt.subplots(15, 15, figsize=(15, 15))

# Flatten the 10x10 grid into a list for easy iteration
axes_flat = axes.flatten()

for i, ax in enumerate(axes_flat):
    latent_vector = torch.tensor(latent_vectors[i], dtype=torch.float32).unsqueeze(0).to(device)
    
    # Decode the latent vector
    with torch.no_grad():
        decoded_img = vae.decode(latent_vector).squeeze().cpu().numpy()

    # If your decoded image is not in the shape you expect, reshape it
    decoded_img = decoded_img.reshape((image_height, image_width))  # Replace with your actual image dimensions
    
    ax.imshow(decoded_img, cmap='gray')  # or cmap='color' if your image is color
    ax.set_title(f'Img {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()


# Testing the Memory Model

In [ ]:
from worldmodels1.src.worldmodels1.mdnrnn import MemoryModel

latent_dim = 32 
action_dim = 3

model = MemoryModel(n_input=latent_dim+action_dim, n_hidden=256, n_gaussians=5, latent_dim=latent_dim)
# Load the state_dict into CPU memory
state_dict = torch.load('./../src/worldmodels1/memory_model.pth', map_location='cpu')


In [ ]:
# dummy input of zeros
dummy_input = torch.zeros(1, 1, latent_dim+action_dim)
#dummy_input = torch.randn(1, 1, latent_dim+action_dim)
print(dummy_input.shape)

# make a forward pass
pi, mu, sigma, _ = model(dummy_input)

In [ ]:
pi.size(), mu.size(), sigma.size()

In [ ]:
def gmm_sample(pi, mu, sigma):
    # Sample one Gaussian index based on pi values
    gaussian_idx = torch.multinomial(pi[0, 0, :], 1)

    # Pick the mu and sigma corresponding to the chosen Gaussian
    chosen_mu = mu[0, 0, gaussian_idx, :]
    chosen_sigma = sigma[0, 0, gaussian_idx, :]

    # Sample a point from the chosen Gaussian distribution
    sample = torch.normal(chosen_mu, chosen_sigma)
    return sample

# Assume pi, mu, sigma have been computed and have the following shapes
# pi: (batch_size, n_gaussians)
# mu: (batch_size, n_gaussians, latent_dim)
# sigma: (batch_size, n_gaussians, latent_dim)
vae.to('cpu')
latent_output = gmm_sample(pi, mu, sigma)
print(latent_output.shape)
print(latent_output)

# decode the latent vector and plot the decoded image
with torch.no_grad():
    decoded_img = vae.decode(latent_output.unsqueeze(0)).squeeze().cpu().numpy()

plt.imshow(decoded_img, cmap='gray')  # or cmap='color' if your image is color


In [ ]:
def mdn_calculate(pi, mu, sigma):
    with torch.no_grad():
        gaussians = torch.distributions.Normal(mu, sigma)
        gaussians = gaussians.sample().squeeze()  # This might still be 2D depending on the original dimensions of mu and sigma
        pi = pi.squeeze()

        # Ensure dimensions are compatible for weighted sum
        pi = pi.view(-1, 1)

        #print(gaussians.shape)
        #print(pi.shape)

        # Use matmul for the weighted sum
        predicted_latent = torch.matmul(pi.T, gaussians)

    return predicted_latent


#test mdn_calculate
predicted_latent = mdn_calculate(pi, mu, sigma)
print(predicted_latent.shape)
print(predicted_latent)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

hidden = None
action1 = widgets.FloatText(value=0.0, description='Action1:')
action2 = widgets.FloatText(value=0.0, description='Action2:')
action3 = widgets.FloatText(value=0.0, description='Action3:')
button = widgets.Button(description="Predict Next Frame")
out = widgets.Output()

vae.to('cpu')
current_state = torch.zeros(1, 1, latent_dim + action_dim)
def on_button_click(b):
    action = torch.Tensor([[[float(action1.value), float(action2.value), float(action3.value)]]])
    #print(action)
    global current_state
    global hidden
    current_state[:, :, -action_dim:] = action
    #print(current_state)
    with torch.no_grad():
        pi, mu, sigma, hidden = model(current_state, hidden)
        latent_output = gmm_sample(pi, mu, sigma)
        #latent_output = mdn_calculate(pi, mu, sigma)
        #print(latent_output)
        current_state = torch.cat((latent_output.unsqueeze(0), action), dim=-1)
        # Assume vae is your VAE model
        decoded_img = vae.decode(latent_output.unsqueeze(0)).squeeze().cpu().numpy()
    with out:
        # Clear the previous frame
        clear_output(wait=True)
        
        # Display the new frame
        plt.imshow(decoded_img, cmap='gray')
        plt.show()

button.on_click(on_button_click)
display(action1, action2, action3, button, out)



# Test Controller

### Load Models


In [27]:
from worldmodels1.src.worldmodels1.cnnvae import VAE
from worldmodels1.src.worldmodels1.mdnrnn import MemoryModel
import torch

latent_dim = 32
action_dim = 3

vae = VAE()
# Load the state_dict into CPU memory
state_dict = torch.load('./../vae2.pth', map_location='cpu')
# Remove 'module.' prefix from state_dict keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
# Load the modified state_dict into the model
vae.load_state_dict(new_state_dict)
vae.eval()

rnn = MemoryModel(n_input=latent_dim+action_dim, n_hidden=256, n_gaussians=5, latent_dim=latent_dim)
# Load the state_dict into CPU memory
state_dict = torch.load('./../src/worldmodels1/memory_model.pth', map_location='cpu')
# Remove 'module.' prefix from state_dict keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
# Load the modified state_dict into the model
rnn.load_state_dict(new_state_dict)
rnn.eval()



MemoryModel(
  (lstm): LSTM(35, 256, batch_first=True)
  (mdn): MDN(
    (z_h): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=325, bias=True)
    )
  )
)

In [29]:
def process_obs(obs, action, vae, rnn, hidden, device='cpu'):
    # obs is 96x96x3 need to convert to 64x64x1. 
    # Convert to PyTorch tensor and normalize and permute dimensions and transfer to device
    obs = torch.from_numpy(obs).permute(2, 0, 1).float().to(device) / 255.0  
    # Rescale and average color channels
    obs = torch.nn.functional.interpolate(obs.unsqueeze(0), size=(64, 64), mode='bilinear', align_corners=False)
    obs = obs.mean(dim=1, keepdim=True)  # Reduce color channels by taking the mean

    with torch.no_grad():
        #print(f'obs shape: {obs.shape}')
        mu, logvar = vae.encode(obs)
        z_t = vae.reparameterize(mu, logvar)
        #print(f'z_t shape: {z_t.shape}')
        # check if action is a tensor
        if not torch.is_tensor(action):
            action = torch.tensor(action).float().to(device).unsqueeze(0)
        rnn_in = torch.cat((z_t, action), dim=1).unsqueeze(0)
        #print(f"obs shape: {obs.shape}")
        #print(f"action shape: {action.shape}")
        #print(f"rnn_in shape: {rnn_in.shape}")
        #print(f"Initial hidden states shapes: {self.hidden[0].shape}, {self.hidden[1].shape}")
        _, _, _, hidden = rnn(rnn_in, hidden)
        # extract hidden state
        h_t = hidden[0]
        #print(f"Final hidden states shapes: {self.hidden[0].shape}, {self.hidden[1].shape}")
        #print(f"h_t shape: {h_t.shape}")
        #print(f"z_t shape: {z_t.shape}")
        # concat z_t and hidden
        z_t = z_t.squeeze(0) # remove batch dimension
        h_t = h_t.squeeze(0).squeeze(0) # remove batch and sequence dimensions
        obs = torch.cat((z_t, h_t))
        #print(f"obs shape: {obs.shape}")
    return obs

In [33]:
# TO DO: plot the reconstructed image and the predicted next image

from stable_baselines3 import PPO
import gymnasium as gym 
from worldmodels1.src.worldmodels1.gym_wrapper import CarRacingWrapper
from IPython import display
import matplotlib.pyplot as plt
import numpy as np

device = 'cpu'

# Load the trained model
controller = PPO.load("./../../ppo_car_racing_56.zip")

# Create the environment
env = gym.make('CarRacing-v2', render_mode='rgb_array')

# Function to display the environment
def show_state(env):
    plt.figure(1)
    plt.clf()
    plt.imshow(env.render())
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)

# Run the trained model
for episode in range(1, 6):  # Run for 5 episodes
    obs, _ = env.reset()
    done = False
    episode_reward = 0
    hidden = (torch.zeros((1, 1, rnn.n_hidden)).to(device), 
               torch.zeros((1, 1, rnn.n_hidden)).to(device))
        # action should be a 1x3 tensor
    action = torch.zeros((1, 3)).to(device)

    
    while not done:
        show_state(env)  # Display the environment
        obs = process_obs(obs, action, vae, rnn, hidden)
        # radnom obs for now of shape (32 +256)
        #obs = np.random.randn(32+256)
        action, _ = controller.predict(obs)
        # random action
        # action = env.action_space.sample()
        # action = action + np.array([0.0, 0.2, -0.45])
        #action[2] = 0.0
        obs, reward, done, _, info = env.step(action)
        episode_reward += reward
        
    print(f"Episode {episode} reward: {episode_reward}")

env.close()


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>